## <font color = blue> Importing Libraries </font>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pt


In [2]:
%matplotlib inline

## <font color = blue> Checkpoint 1 and Table 1.1</font>

####  <font color = blue>Data Importing from Companies and Rounds2 </font>

1. Load Companies data from CSV file or directly from the link.
2. Load rounds2 data from CSV.

In [3]:
companies=pd.read_csv("companies.txt",sep="\t",\
                      encoding='latin1')

rounds2=pd.read_csv('rounds2.csv',encoding='latin1')


Unique count for Companies after removing non readable data 

In [4]:
companies['permalink']=companies['permalink'].str.replace(r'[^\x00-\x7F]+', '').str.upper()
companies['permalink'].nunique()

66368

Unique count for rounds2 file remove non readable data and leading and trailing spaces from companies name from companies name

In [5]:
rounds2['company_permalink']=rounds2['company_permalink'].str.replace(r'[^\x00-\x7F]+', '').str.upper()
rounds2['company_permalink'].nunique()

66368

####  <font color = blue>Creating master_frame</font>

Not dropping rows with blank <b>"raised_amount_usd" or "funding_round_type"</b>

In [6]:
#Not dropping rows with blank "raised_amount_usd or funding_round_type"
master_frame=rounds2.merge(companies,how='inner',right_on='permalink',left_on='company_permalink')


##### Total count of the master_frame

In [7]:
master_frame.shape

(114946, 16)

Cleaning the master_Frame
1. Removing all the rows where raised_Amount_usd is blank. Cant fill with mean/mode/median as the min and max difference is huge.
2. Removing all the rows where category_list if blank as we cant fill the data with category.
3. Removing rows where country_code is blank as filling the data is not possible, filling with highest occurrence of country is not appropriate.

In [8]:
master_frame=master_frame[master_frame['raised_amount_usd'].notnull()]
master_frame=master_frame[master_frame['category_list'].notnull()]
master_frame=master_frame[master_frame['country_code'].notnull()]

Setting number format to 2 decimal places

In [9]:
pd.set_option('display.float_format','{:,.2f}'.format)


## <font color = blue> Checkpoint 2 and Table 2.1</font>

1. Average funding amount of venture type
2. Average funding amount of angel type
3. Average funding amount of seed type
4. Average funding amount of private equity type
5. Considering that Spark Funds wants to invest between 5 to 15 million USD per  investment round, which investment type is the most suitable for them?

In [10]:
# create rep_value data frame to Average amount of each funding type

rep_value=master_frame.groupby('funding_round_type').agg({'raised_amount_usd':'mean'})
rep_value.reset_index(inplace=True)
rep_value.rename(columns={'raised_amount_usd':'Average_Amount'},inplace=True)

rep_value[rep_value.funding_round_type.isin(['venture','angel','private_equity','seed'])]\
.sort_values('Average_Amount',ascending=False)



,funding_round_type,Average_Amount
8,private_equity,"73,938,486.28"
13,venture,"11,724,222.69"
0,angel,"971,573.89"
11,seed,"747,793.68"


## <font color = blue> Checkpoint 3 and Table 3.1</font>

1. Top English speaking country
2. Second English speaking country
3. Third English speaking country


In [11]:
top9=master_frame.loc[ (master_frame['funding_round_type']== 'venture') &\
                      ~(master_frame['country_code'].isnull())& (master_frame['raised_amount_usd'].notnull()) 
                      ,\
                      ['country_code','raised_amount_usd']]\
                    .groupby(by='country_code').agg('sum').sort_values('raised_amount_usd',ascending=False).head(9)

Creating dataframe for english speaking countries with country codes

In [12]:
dic={'COUNTRY':["Antigua and Barbuda","Australia","The Bahamas","Barbados","Belize","Botswana","Cameroon","Canada","Dominica","Eritrea","Ethiopia","Fiji","The Gambia","Ghana","Grenada","Guyana","India","Ireland","Jamaica","Kenya","Kiribati","Lesotho","Liberia","Malawi","Malta","Marshall Islands","Mauritius","Federated States of Micronesia","Namibia","Nauru","New Zealand","Nigeria","Pakistan","Palau","Papua New Guinea","Philippines","Rwanda","Saint Kitts and Nevis","Saint Lucia","Saint Vincent and the Grenadines","Samoa","Seychelles","Sierra Leone","Singapore","Solomon Islands","South Africa","South Sudan","Sudan","Swaziland","Tanzania","Tonga","Trinidad and Tobago","Tuvalu","Uganda","United Kingdom","United States","Vanuatu","Zambia","Zimbabwe"]\
    ,'A3':["ATG","AUS","BHS","BRB","BLZ","BWA","CMR","CAN","DMA","ERI","ETH","FJI","GMB","GHA","GRD","GUY","IND","IRL","JAM","KEN","KIR","LSO","LBR","MWI","MLT","MHL","MUS","FSM","NAM","NRU","NZL","NGA","PAK","PLW","PNG","PHL","RWA","KNA","LCA","VCT","WSM","SYC","SLE","SGP","SLB","ZAF","SSD","SDN","SWZ","TZA","TON","TTO","TUV","UGA","GBR","USA","VUT","ZMB","ZWE"],'Language':["English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English","English"]}

eng_offcal_lang=pd.DataFrame(dic)


In [13]:
# top 9 countries based on FT as venrture
top9

,raised_amount_usd
country_code,
USA,"420,068,029,342.00"
CHN,"39,338,918,773.00"
GBR,"20,072,813,004.00"
IND,"14,261,508,718.00"
CAN,"9,482,217,668.00"
FRA,"7,226,851,352.00"
ISR,"6,854,350,477.00"
DEU,"6,306,921,981.00"
JPN,"3,167,647,127.00"


Finding top 3 English speaking countries

In [14]:
top_english_countries=top9.merge(eng_offcal_lang,left_on='country_code',right_on='A3')\
            .sort_values('raised_amount_usd',ascending=False).head(3)
top_english_countries

,raised_amount_usd,COUNTRY,A3,Language
0,"420,068,029,342.00",United States,USA,English
1,"20,072,813,004.00",United Kingdom,GBR,English
2,"14,261,508,718.00",India,IND,English


## <font color = blue> Checkpoint 4</font>

1. creating primary_sector column in master_frame

In [15]:
master_frame['primary_sector']=master_frame['category_list'].str.split('|',1,True).iloc[:,0]

Working with mapping file
1. Extracting the data in mapping dataframe
2. Data cleaning, removing 0's with na from category_list columns
3. Unpivoting the datafram using 'melt' function in dataframe. Name sector as 'main_sector' and 0&1 value column as Yes/No
4. Create new data frame 'main_sector' and fill it with category and main_sector i.e. values as 1 in Yes/No column of melting data frame.



In [16]:
mapping=pd.read_csv('mapping.csv',encoding='latin1')

mapping['category_list']=mapping['category_list'].str.replace('0','na')
melting=mapping.melt(id_vars='category_list',var_name='main_sector',value_name='Yes/No')

main_sector=melting.loc[(melting['Yes/No']==1),['category_list','main_sector','Yes/No']]
main_sector.head()


Merging master_frame and main_sector

1. convert primary_sector columns of master_frame to upper case.
2. convert category_list columns of main_sector to upper case.
3. Merger both the dataframes.
4. Dropping columns 'category_list_y', which got created during the merging from master_frame

In [17]:
master_frame['primary_sector']=master_frame['primary_sector'].str.upper()
main_sector['category_list']=main_sector['category_list'].str.upper()
master_frame=master_frame.merge(main_sector[['category_list','main_sector']],\
                                          how='inner',left_on='primary_sector',right_on='category_list')

master_frame.drop('category_list_y',axis=1,inplace=True)
master_frame.rename(columns={'category_list_x':'category_list'},inplace=True)

## <font color = blue> Checkpoint 5 and Table 5.1</font>

#### <font color=blue >Creating D1, D2 and D3</font>
1. Create D1 with filter: country_code=USA , funding_round_type= ventire,category_list as not null, and raised_amount_usd  between 5 million to 15million inclding both boundaries.
2. Create D2 with filter: country_code=USA , funding_round_type= ventire,category_list as not null,and raised_amount_usd  between 5 million to 15million inclding both boundaries.
3. Create D3 with filter: country_code=USA , funding_round_type= ventire,category_list as not null,and raised_amount_usd  between 5 million to 15million inclding both boundaries.

In [18]:
#(master_frame['raised_amount_usd'].between(5000000,15000000))& 
#master_frame.to_csv('wiht primary sector2.csv')
D1=master_frame[(master_frame['country_code']== 'USA') & (master_frame['funding_round_type']== 'venture')&\
                (master_frame['raised_amount_usd'].between(5000000,15000000))\
                & (master_frame['category_list'].notnull())]



D2=master_frame[(master_frame['country_code']== 'GBR') & (master_frame['funding_round_type']== 'venture')&\
             (master_frame['raised_amount_usd'].between(5000000,15000000))& (master_frame['category_list'].notnull())]



D3=master_frame[(master_frame['country_code']== 'IND') & (master_frame['funding_round_type']== 'venture')&\
             (master_frame['raised_amount_usd'].between(5000000,15000000))& master_frame['category_list'].notnull()]


In [19]:
# Adding total number (or count) of investments for each main sector in a separate column
# Adding total amount invested in each main sector in a separate column
D1=D1.merge(D1.groupby('main_sector').agg({'main_sector':'count','raised_amount_usd':'sum'})\
            .rename(columns={'main_sector':'total_number','raised_amount_usd':'total amount invested'})\
            .reset_index(),how='inner',left_on='main_sector',right_on='main_sector')

D2=D2.merge(D2.groupby('main_sector').agg({'main_sector':'count','raised_amount_usd':'sum'}).\
            rename(columns={'main_sector':'total_number','raised_amount_usd':'total amount invested'})\
            .reset_index(),how='inner',left_on='main_sector',right_on='main_sector')



D3=D3.merge(D3.groupby('main_sector').agg({'main_sector':'count','raised_amount_usd':'sum'}).\
            rename(columns={'main_sector':'total_number','raised_amount_usd':'total amount invested'})\
            .reset_index(),how='inner',left_on='main_sector',right_on='main_sector')


Total number of Investments (count)

In [20]:
print("D1: "+ str(D1['main_sector'].count()))
print("D2: "+ str(D2['main_sector'].count()))
print("D3: "+ str(D3['main_sector'].count()))

D1: 12063
D2: 621
D3: 328


Total amount of investment (USD)

In [21]:
print("D1: " + str(D1['raised_amount_usd'].sum()))
print("D2: " + str(D2['raised_amount_usd'].sum()))
print("D3: " + str(D3['raised_amount_usd'].sum()))

D1: 107757097294.0
D2: 5379078691.0
D3: 2949543602.0


Top **3** Sector name (no. of investment-wise)

In [22]:
# Top 3 sectors for D1
country_1=D1.groupby('main_sector').agg({'total_number':'max'}).sort_values('total_number',ascending=False).head(3)
country_1

,total_number
main_sector,
Others,2950
"Social, Finance, Analytics, Advertising",2714
Cleantech / Semiconductors,2350


In [23]:
# Top 3 sectors for D2
country_2=D2.groupby('main_sector').agg({'total_number':'max'}).sort_values('total_number',ascending=False).head(3)
country_2

,total_number
main_sector,
Others,147
"Social, Finance, Analytics, Advertising",133
Cleantech / Semiconductors,130


In [24]:
# Top 3 sectors for D3
country_3=D3.groupby('main_sector').agg({'total_number':'max'}).sort_values('total_number',ascending=False).head(3)
country_3

,total_number
main_sector,
Others,110
"Social, Finance, Analytics, Advertising",60
"News, Search and Messaging",52


For point 3 (top sector count-wise) for D1,D2 and D3 frame, which company received the highest investment?

In [25]:
print("Top Sector (count wise), received highest investment in D1:\n " + \
      str(D1.loc[D1['main_sector']=='Others',['name','raised_amount_usd']]\
     .groupby('name').agg({'raised_amount_usd':'sum'}).sort_values('raised_amount_usd',ascending=False).head(1)))

print("\n Top Sector (count wise) ,received highest investment in D2:\n " + \
      str(D2.loc[D2['main_sector']=='Others',['name','raised_amount_usd']]\
     .groupby('name').agg({'raised_amount_usd':'sum'}).sort_values('raised_amount_usd',ascending=False).head(1)))

print("\n Top Sector (count wise), received highest investment in D3:\n " + \
      str(D3.loc[D3['main_sector']=='Others',['name','raised_amount_usd']]\
     .groupby('name').agg({'raised_amount_usd':'sum'}).sort_values('raised_amount_usd',ascending=False).head(1)))

Top Sector (count wise), received highest investment in D1:
              raised_amount_usd
name                          
Virtustream      64,300,000.00

 Top Sector (count wise) ,received highest investment in D2:
                 raised_amount_usd
name                             
Electric Cloud      37,000,000.00

 Top Sector (count wise), received highest investment in D3:
               raised_amount_usd
name                           
FirstCry.com      39,000,000.00


For point 3 (second top sector count-wise) for D1,D2 and D3 frame, which company received the highest investment?

In [26]:
print("Second Top Sector (count wise), received highest investment in D1:\n\n " + \
      str(D1.loc[D1['main_sector']=='Social, Finance, Analytics, Advertising',['name','raised_amount_usd']]\
     .groupby('name').agg({'raised_amount_usd':'sum'}).sort_values('raised_amount_usd',ascending=False).head(1)))

print("\n Second Top Sector (count wise) ,received highest investment in D2:\n\n " + \
      str(D2.loc[D2['main_sector']=='Social, Finance, Analytics, Advertising',['name','raised_amount_usd']]\
     .groupby('name').agg({'raised_amount_usd':'sum'}).sort_values('raised_amount_usd',ascending=False).head(1)))

print("\n Second Top Sector (count wise), received highest investment in D3:\n\n " + \
      str(D3.loc[D3['main_sector']=='Social, Finance, Analytics, Advertising',['name','raised_amount_usd']]\
     .groupby('name').agg({'raised_amount_usd':'sum'}).sort_values('raised_amount_usd',ascending=False).head(1)))

Second Top Sector (count wise), received highest investment in D1:

                                  raised_amount_usd
name                                              
SST Inc. (Formerly ShotSpotter)      67,933,006.00

 Second Top Sector (count wise) ,received highest investment in D2:

                        raised_amount_usd
name                                    
Celltick Technologies      37,500,000.00

 Second Top Sector (count wise), received highest investment in D3:

                  raised_amount_usd
name                              
Manthan Systems      50,700,000.00


## <font color=blue> PLOT 1: </font>
Total investments (globally) in venture, seed, and private equity, and the average amount of investment in each funding type

In [27]:
#rep_value[rep_value['funding_round_type'].isin(['venture','angel','private_equity','seed'])]
Total_invest=master_frame.groupby('funding_round_type').agg({'raised_amount_usd':'sum'})
Total_invest.reset_index(inplace=True)

Total_invest=(Total_invest[Total_invest.funding_round_type.isin(['venture','angel','private_equity','seed'])])
inves=master_frame['raised_amount_usd'].sum()
Total_invest['Percentage_investment']=100*(Total_invest['raised_amount_usd']/inves)
Total_invest.drop('raised_amount_usd',axis=1,inplace=True)
Total_invest.rename(columns={'Average_amount':'Average_investment'})
#rep_value.columns
Total_invest=Total_invest.merge(rep_value,left_on='funding_round_type',right_on='funding_round_type',how='inner')

pt.figure(figsize=(15,5))
pt.subplot(1, 2, 1)
pt.pie(x=Total_invest['Percentage_investment'],labels=Total_invest['funding_round_type'])
pt.title('Fraction of total investments (globally) in in each Funding type')
#pt.legend()
pt.subplot(1, 2,2)
pt.bar(x=Total_invest['funding_round_type'],height=Total_invest['Average_Amount'])
pt.title('Average amount of investment in each funding type')
pt.show()
#Total_invest
#ax.legend(bbox_to_anchor=(1.2, 0.5))

## <font color=blue> PLOT 2: </font>

A plot showing the top 9 countries against the total amount of investments of funding type FT. 

top9['%']=(100*top9.raised_amount_usd/top9.raised_amount_usd.sum())
top9
pt.figure(figsize=(15,5))
pt.subplot(1, 2, 1)
pt.bar(x=top9.reset_index()['country_code'],height=top9['%'])
pt.title('Top 9 country as per Investment')
#pt.legend()
pt.subplot(1, 2,2)
pt.bar(x=top_english_countries['COUNTRY'],height=top_english_countries['raised_amount_usd'])
pt.title('Top 3 English speaking countries')

## <font color=blue> PLOT 3: </font>
A plot showing the number of investments in the top 3 sectors of the top 3 countries on one chart (for the chosen investment type FT). 


result=pd.concat([country_1.T,country_2.T,country_3.T],keys=['USA','GBR','INR'],sort=False).reset_index(level=1).drop(columns='level_1')
result.plot.bar(figsize=(8,6),legend='reverse',rot=0,title='Top 3 sectors of Top 3 Countries')
pt.show()